In [14]:

from dotenv import load_dotenv
import pandas
import os
import psycopg2 as ps
import requests
import json
import pandas as pd
import urllib3
import sqlalchemy

pd.set_option('display.max_columns', None)
load_dotenv()

print(os.environ["PSQL_DB_HOST"])
print(os.environ["PSQL_DB_USER"])
conn = ps.connect(
    host=os.environ["PSQL_DB_HOST"],
    port=os.environ["PSQL_DB_PORT"],
    dbname=os.environ["PSQL_DB_NAME"],
    user=os.environ["PSQL_DB_USER"],
    password=os.environ["PSQL_DB_PASS"])


def cubesql(query):
    return pandas.read_sql(query, conn)


print(" successfully connected")

sqlalchemy.create_engine(os.environ["DATABASE_URL"])

%load_ext sql

print(" inline sql successfully connected")


127.0.0.1
crowd
 successfully connected
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 inline sql successfully connected


In [2]:
query = """
select org.id, org.name 
  from organizations org
  where org."tenantId" = 'ccff5355-cf54-40a1-9a2e-8e4a447ae73a'
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_43045/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,id,name
0,a0f28676-f608-47a1-830a-62ebf066a0ef,lfph @cncf @coreinfrastructure
1,3a091dd3-23ad-49f6-88bc-7024168d4d44,Stacklet
2,4060cdf1-d562-40e8-b4bc-a0655b619e94,"jenkinsci, @keptn, @cdfoundation"
3,d45f7a47-3daf-40e7-9da1-a8d4e370a846,"cncf, the Linux Foundation"
4,5a01ac2e-ebb6-4f62-88b7-b2de30375533,Replane
...,...,...
63,493a7228-09fa-4f2c-a663-78759198bfba,"Lukina, Inc."
64,893bdff7-eedf-4827-a305-4b5d49c1536c,freelance iOS and Web developer
65,6db0e468-b60a-4d8b-aeaf-e5deb64aa0d1,Uniqud
66,f859d3f6-d097-4264-a7ce-0da7acab3f1d,"Iptalent Consulting Co., LTD."


In [26]:
query = """
with org_members as (
  select org.id, org.name ,mor."memberId"
  from organizations org
  left join "memberOrganizations" mor on mor."organizationId" = org.id
  where org."tenantId" = 'ccff5355-cf54-40a1-9a2e-8e4a447ae73a'
)
select name org_name, count("memberId") memberscount from org_members group by name order by  memberscount desc
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_28690/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,org_name,memberscount
0,The Linux Foundation,19
1,Google,5
2,Cloud Native Computing Foundation,3
3,Linux Foundation Engineering,2
4,Microsoft Azure,2
...,...,...
63,Replane,0
64,Call for Code® with The Linux Foundation,0
65,Microsoft Corporation,0
66,"jenkinsci, @keptn, @cdfoundation",0


In [27]:
query = """
select "organizationId","memberId" from "memberOrganizations"
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_28690/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,organizationId,memberId
0,611531cd-9a9c-4686-ab57-d90fc1905fe9,dc0d1a0b-9cc0-4a21-937f-7593e3ddbe0c
1,611531cd-9a9c-4686-ab57-d90fc1905fe9,7e3c2f14-0afc-43ef-bbaa-2c5f0919b9f2
2,8553678b-2dc3-4ee1-810b-b5da5f40a400,601a60d0-6167-4071-8bc0-307aa429a4a4
3,611531cd-9a9c-4686-ab57-d90fc1905fe9,fcdbce6f-234d-4e20-bb00-c136d48e9afd
4,b27d04aa-fedf-4284-a774-1e0dae60c2f0,63880f02-6030-44fb-bfc8-eeec51d97c94
...,...,...
358,7d2d3c2c-9826-40a3-a8e8-2b999adc3cae,ed4c4a1c-c18c-4d8c-ad82-10c8093a36d6
359,6401567d-9b7a-4f3e-9b83-58ddbd21c35a,a09c2cd5-ce1a-47ae-9da1-71e684140a08
360,7d2d3c2c-9826-40a3-a8e8-2b999adc3cae,cc49199d-102b-4b75-9c4b-4d2211027de2
361,7d2d3c2c-9826-40a3-a8e8-2b999adc3cae,594ddd07-365e-4f4c-9f17-cf7e3c25d3bb


In [28]:
query = """
select mm."organizationId", oo.name , count(*)
from activities aa
left join "memberOrganizations" mm on mm."memberId" = aa."memberId"
left join "organizations" oo on oo.id = mm."organizationId"
group by 1,2
order by 3 desc
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_28690/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,organizationId,name,count
0,1ee58b13-b959-411a-8af0-ce2b236bcf26,The Linux Foundation,6793
1,None,None,6286
2,611531cd-9a9c-4686-ab57-d90fc1905fe9,crowd.dev,1458
3,e22c9f3f-d69b-423d-afa1-dbf2d962df9b,Linux Foundation Engineering,1103
4,0458e046-239b-4d7a-8046-9b635221a130,proximabiz,627
...,...,...,...
304,900eafa7-642b-4a7a-9327-8bb1898b945b,PingCAP,1
305,305a1631-7aec-4ab7-b055-410fde8379ee,Dozer Data,1
306,a8187285-b124-4b7c-85ae-672600ed838b,Cobeia,1
307,a0ef73c5-404c-469d-9a8b-77adbf56de64,Harvard GAMI,1


In [19]:
query = """
select *
from "organizations" oo 
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_28690/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,id,name,url,description,parentUrl,emails,phoneNumbers,logo,tags,twitter,linkedin,crunchbase,employees,revenueRange,importHash,createdAt,updatedAt,deletedAt,tenantId,createdById,updatedById,location,github,website,isTeamOrganization
0,611531cd-9a9c-4686-ab57-d90fc1905fe9,crowd.dev,https://github.com/CrowdDotDev,Open-source community and data tools built to ...,None,None,None,https://avatars.githubusercontent.com/u/855519...,None,{'handle': 'CrowdDotDev'},None,None,None,None,None,2023-04-03 12:56:35.782000+00:00,2023-04-03 12:56:35.782000+00:00,None,2c6ac1bd-a302-44e1-b6ba-79ef8b2a5167,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,None,{'handle': 'CrowdDotDev'},https://www.crowd.dev/,False
1,8553678b-2dc3-4ee1-810b-b5da5f40a400,Flipt Software,None,None,None,None,None,None,None,None,None,None,None,None,None,2023-04-03 12:58:04.159000+00:00,2023-04-03 12:58:04.159000+00:00,None,2c6ac1bd-a302-44e1-b6ba-79ef8b2a5167,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,None,None,None,False
2,b27d04aa-fedf-4284-a774-1e0dae60c2f0,Antosser Inc.,None,None,None,None,None,None,None,None,None,None,None,None,None,2023-04-03 12:58:11.418000+00:00,2023-04-03 12:58:11.418000+00:00,None,2c6ac1bd-a302-44e1-b6ba-79ef8b2a5167,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,None,None,None,False
3,a2a14f4d-f32a-47bf-9577-5227dacb1d43,Saleor Commerce,https://github.com/saleor,The commerce API that puts developers first.,None,None,None,https://avatars.githubusercontent.com/u/137961...,None,{'handle': 'getsaleor'},None,None,None,None,None,2023-04-03 12:58:20.059000+00:00,2023-04-03 12:58:20.059000+00:00,None,2c6ac1bd-a302-44e1-b6ba-79ef8b2a5167,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,None,{'handle': 'saleor'},https://saleor.io/,False
4,bb07d5a3-a051-4dfb-a164-d7cae825f176,Kestra,https://github.com/kestra-io,An infinitely scalable opensource orchestratio...,None,None,None,https://avatars.githubusercontent.com/u/590333...,None,{'handle': 'kestra_io'},None,None,None,None,None,2023-04-03 12:58:20.271000+00:00,2023-04-03 12:58:20.271000+00:00,None,2c6ac1bd-a302-44e1-b6ba-79ef8b2a5167,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,a9768a86-2673-4c0f-ad4b-da0da4e4e59e,France,{'handle': 'kestra-io'},https://kestra.io,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,f859d3f6-d097-4264-a7ce-0da7acab3f1d,"Iptalent Consulting Co., LTD.",None,None,None,None,None,None,None,None,None,None,None,None,None,2023-04-04 18:56:53.033000+00:00,2023-04-27 03:48:49.593000+00:00,None,ccff5355-cf54-40a1-9a2e-8e4a447ae73a,6f8ea61c-1b19-4658-9dd6-d1716d32036b,4514f84b-8a27-478e-86f5-53904c6b5520,None,None,None,False
313,99691be2-fa01-44e8-87a4-7fd807a5afc1,Syndio,https://github.com/syndio,,None,None,None,https://avatars.githubusercontent.com/u/785368...,None,None,None,None,None,None,None,2023-04-04 18:56:53.351000+00:00,2023-04-27 03:48:49.707000+00:00,None,ccff5355-cf54-40a1-9a2e-8e4a447ae73a,6f8ea61c-1b19-4658-9dd6-d1716d32036b,4514f84b-8a27-478e-86f5-53904c6b5520,None,{'handle': 'syndio'},https://synd.io,False
314,7d2d3c2c-9826-40a3-a8e8-2b999adc3cae,crowd.dev,https://github.com/CrowdDotDev,Open-source community and data tools built to ...,None,None,None,https://avatars.githubusercontent.com/u/855519...,None,{'handle': 'CrowdDotDev'},None,None,None,None,None,2023-04-27 07:31:42.978000+00:00,2023-04-27 07:31:42.978000+00:00,None,c3e2c6ec-76a3-4778-a096-37955f3bff51,2e0fdc23-792f-4179-8aeb-b0aac81443a3,2e0fdc23-792f-4179-8aeb-b0aac81443a3,None,{'handle': 'CrowdDotDev'},https://www.crowd.dev/,False
315,81341660-7487-4572-ad51-7acbae689803,Billogram – billogram,None,None,None,None,None,None,None,None,None,None,None,None,None,2023-04-27 13:23:11.048000+00:00,2023-04-27 13:23:11.048000+00:00,None,c3e2c6ec-76a3-4778-a096-37955f3bff51,2e0fdc23-792f-4179-8aeb-b0aac81443a3,2e0fdc23-792f-4179-8aeb-b0aac81443a3,None,None,None,Fa

In [15]:
query = """
SELECT
    "organizations".name "organizations__name",
    count("activities".id) "activities__count", 
    sum(count("activities".id)) OVER () "activities__count__sum"
  FROM
    activities AS "activities"
    LEFT JOIN (
      select
        "organizationId",
        "memberId"
      from
        "memberOrganizations"
    ) AS "member_organizations" ON "activities"."memberId" = "member_organizations"."memberId"
    LEFT JOIN organizations AS "organizations" ON "member_organizations"."organizationId" = "organizations".id
  WHERE
    (1=1
    )
    AND ("activities"."tenantId" = 'ccff5355-cf54-40a1-9a2e-8e4a447ae73a')
    AND ("organizations".name IS NOT NULL)
  GROUP BY
    1
  ORDER BY
    2 DESC
"""
response = cubesql(query)
df = pd.DataFrame(response)
df


/var/folders/r9/sb15zqmn7f9197191j5zs39h0000gn/T/ipykernel_13965/3795606648.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pandas.read_sql(query, conn)


,organizations__name,activities__count,activities__count__sum
0,The Linux Foundation,6830,8882.0
1,Linux Foundation Engineering,1114,8882.0
2,proximabiz,627,8882.0
3,Mckinsey & Company,75,8882.0
4,Snyk,66,8882.0
...,...,...,...
58,Atlassian,1,8882.0
59,AppDynamics,1,8882.0
60,JFrog Ltd.,1,8882.0
61,Arteaga Inc.,1,8882.0


In [16]:
%%sql
SELECT
    "organizations".name "organizations__name",
    count("activities".id) "activities__count", 
    sum(count("activities".id)) OVER () "activities__count__sum"
  FROM
    activities AS "activities"
    LEFT JOIN (
      select
        "organizationId",
        "memberId"
      from
        "memberOrganizations"
    ) AS "member_organizations" ON "activities"."memberId" = "member_organizations"."memberId"
    LEFT JOIN organizations AS "organizations" ON "member_organizations"."organizationId" = "organizations".id
  WHERE
    (1=1
    )
    AND ("activities"."tenantId" = 'ccff5355-cf54-40a1-9a2e-8e4a447ae73a')
    AND ("organizations".name IS NOT NULL)
  GROUP BY
    1
  ORDER BY
    2 DESC

63 rows affected.


organizations__name,activities__count,activities__count__sum
The Linux Foundation,6830,8882
Linux Foundation Engineering,1114,8882
proximabiz,627,8882
Mckinsey & Company,75,8882
Snyk,66,8882
Google,40,8882
Oracle,16,8882
Red Hat,11,8882
Cloud Foundry,10,8882
VMware,6,8882
